In [890]:
import logging
from itertools import combinations
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import random
from icecream import ic
import heapq

I have add some properties on the class problem to make it easier access to some data

In [891]:
class Problem:
    _graph: nx.Graph
    _alpha: float
    _beta: float
    _density: float

    def __init__(
        self,
        num_cities: int,
        *,
        alpha: float = 1.0,
        beta: float = 1.0,
        density: float = 0.5,
        seed: int = 42,
    ):
        rng = np.random.default_rng(seed)
        self._alpha = alpha
        self._beta = beta
        self._density = density
        cities = rng.random(size=(num_cities, 2))
        cities[0, 0] = cities[0, 1] = 0.5

        self._graph = nx.Graph()
        self._graph.add_node(0, pos=(cities[0, 0], cities[0, 1]), gold=0)
        for c in range(1, num_cities):
            self._graph.add_node(c, pos=(cities[c, 0], cities[c, 1]), gold=(1 + 999 * rng.random()))

        tmp = cities[:, np.newaxis, :] - cities[np.newaxis, :, :]
        d = np.sqrt(np.sum(np.square(tmp), axis=-1))
        for c1, c2 in combinations(range(num_cities), 2):
            if rng.random() < density or c2 == c1 + 1:
                self._graph.add_edge(c1, c2, dist=d[c1, c2])

        assert nx.is_connected(self._graph)

    @property
    def graph(self) -> nx.Graph:
        return nx.Graph(self._graph)

    @property
    def dist_dict(self):
        return {(u, v): data['dist'] for u, v, data in self._graph.edges(data=True)}

    @property
    def gold_dict(self):
        return {n: data['gold'] for n, data in self._graph.nodes(data=True)}

    def cost(self, path, weight):
        dist = nx.path_weight(self._graph, path, weight='dist')
        return dist + (self._alpha * dist * weight) ** self._beta

    def baseline(self):
        total_cost = 0
        for dest, path in nx.single_source_dijkstra_path(
            self._graph, source=0, weight='dist'
        ).items():
            cost = 0
            for c1, c2 in zip(path, path[1:]):
                cost += self.cost([c1, c2], 0)
                cost += self.cost([c1, c2], self._graph.nodes[dest]['gold'])
            logging.debug(
                f"dummy_solution: go to {dest} ({' > '.join(str(n) for n in path)} ({cost})"
            )
            total_cost += cost
        return total_cost

    def plot(self):
        plt.figure(figsize=(10, 10))
        pos = nx.get_node_attributes(self._graph, 'pos')
        size = [100] + [self._graph.nodes[n]['gold'] for n in range(1, len(self._graph))]
        color = ['red'] + ['lightblue'] * (len(self._graph) - 1)
        return nx.draw(self._graph, pos, with_labels=True, node_color=color, node_size=size)

In [892]:
logging.getLogger().setLevel(logging.WARNING)

ic(Problem(100, density=0.2, alpha=1, beta=1).baseline())
ic(Problem(100, density=0.2, alpha=2, beta=1).baseline())
ic(Problem(100, density=0.2, alpha=1, beta=2).baseline())
ic(Problem(100, density=1, alpha=1, beta=1).baseline())
ic(Problem(100, density=1, alpha=2, beta=1).baseline())
ic(Problem(100, density=1, alpha=1, beta=2).baseline())
ic(Problem(1_000, density=0.2, alpha=1, beta=1).baseline())
ic(Problem(1_000, density=0.2, alpha=2, beta=1).baseline())
ic(Problem(1_000, density=0.2, alpha=1, beta=2).baseline())
ic(Problem(1_000, density=1, alpha=1, beta=1).baseline())
ic(Problem(1_000, density=1, alpha=2, beta=1).baseline())
ic(Problem(1_000, density=1, alpha=1, beta=2).baseline())

None

ic| Problem(100, 

density=0.2, alpha=1, beta=1).baseline(): np.float64(25266.40561851072)
ic| Problem(100, density=0.2, alpha=2, beta=1).baseline(): np.float64(50425.30961817918)
ic| Problem(100, density=0.2, alpha=1, beta=2).baseline(): np.float64(5334401.927002504)
ic| Problem(100, density=1, alpha=1, beta=1).baseline(): np.float64(18266.18579582672)
ic| Problem(100, density=1, alpha=2, beta=1).baseline(): np.float64(36457.918462372065)
ic| Problem(100, density=1, alpha=1, beta=2).baseline(): np.float64(5404978.08899582)
ic| Problem(1_000, density=0.2, alpha=1, beta=1).baseline(): np.float64(195402.95810394012)
ic| Problem(1_000, density=0.2, alpha=2, beta=1).baseline(): np.float64(390028.72126288974)
ic| Problem(1_000, density=0.2, alpha=1, beta=2).baseline(): np.float64(37545927.70213464)
ic| Problem(1_000, density=1, alpha=1, beta=1).baseline(): np.float64(192936.23377726765)
ic| Problem(1_000, density=1, alpha=2, beta=1).baseline(): np.float64(385105.64149576554)
ic| Problem(1_000, density=1, alph

#### COST FUNCTION


In [893]:
def calculate_full_path_cost_final(problem_instance, path, trip_counts=None):
    
    total_cost = 0.0
    current_weight = 0.0
    infeasible_nodes = []
    
    # Extract distance and gold data from problem
    dist_dict = problem_instance.dist_dict
    gold_dict = problem_instance.gold_dict
    
    # Count number of trips (returns to depot) in path
    total_trips_in_path = 0
    for i in range(len(path) - 1):
        if path[i+1][0] == 0:
            total_trips_in_path += 1
    
    # Use default trip counts if not provided
    if trip_counts is None:
        trip_counts = [1] * total_trips_in_path
    
    trip_index = 0
    current_trip_count = trip_counts[trip_index] if trip_index < len(trip_counts) else 1
    
    # Traverse path and accumulate costs
    for i in range(len(path) - 1):
        u = path[i][0]
        v = path[i + 1][0]
        u_flag = path[i][1]
        
        this_segment_trips = current_trip_count

        # Accumulate gold weight if node is active
        if u != 0 and u_flag == True:
            gold_at_u = gold_dict.get(u, 0)
            current_weight += gold_at_u / this_segment_trips
        
        # Get edge distance
        dist_uv = dist_dict.get((min(u, v), max(u, v)))
        if dist_uv is None:
            dist_uv = 1.0
        
        # Compute segment cost with weight penalty
        cost_segment_unit = dist_uv + (problem_instance._alpha * dist_uv * current_weight) ** problem_instance._beta
        total_cost += cost_segment_unit * this_segment_trips

        # Reset weight at depot and advance trip counter
        if v == 0:
            current_weight = 0.0
            
            trip_index += 1
            if trip_index < len(trip_counts):
                current_trip_count = trip_counts[trip_index]
            else:
                current_trip_count = 1
        
        # Reset weight at start node        
        if u == 0:
            current_weight = 0.0
    
    return (len(infeasible_nodes), total_cost)

#### CREATION OF THE POPOLUTATION

I have create a list that contain all the shortest path to arrive to a certain node. I randomly choose which node visit. After visiting him, i take the gold from the node and if there are other city where i haven't take the gold yet I also take the gold from there. In this way I can obtain a greedy population with different individuals

In [894]:
def build_nearest_neighbors_cache(P, k=5):
   #data structure to hold the k nearest neighbors for each node
    dist_dict = P.dist_dict
    graph = P.graph
    distance_near_cache = {}
    
 
    for node in graph.nodes():
        neighbors_with_dist = []
        for (u, v), dist in dist_dict.items():
            if u == node:
                neighbors_with_dist.append((v, dist))
            elif v == node:
                neighbors_with_dist.append((u, dist))
        
        neighbors_with_dist.sort(key=lambda x: x[1])
        distance_near_cache[node] = {
            neighbor: dist for neighbor, dist in neighbors_with_dist[:k]
        }
    
    return distance_near_cache



In [895]:
def neighborhood_greedy_strategy_dijistra(problem_instance):
    graph = problem_instance.graph
    
    # Compute shortest paths from depot (node 0) to all nodes using Dijkstra
    node_to_path = dict(nx.single_source_dijkstra_path(graph, source=0, weight='dist'))

    # Create ordered list of paths sorted by node ID
    path_list = [node_to_path[node] for node in sorted(node_to_path.keys())]

    return path_list

In [896]:
def choice_a_path(path_list, seed=42):
    # Initialize empty path and RNG
    full_path = []
    rng = random.Random(seed)
    
    # Track nodes not yet visited for gold collection
    ungolden_nodes = [i for i in range(1, len(path_list))]
    
    # Build path by randomly selecting unvisited nodes
    while ungolden_nodes:
        # Randomly choose next node to visit
        node = rng.choice(ungolden_nodes)
        
        # Get shortest path from depot to chosen node
        path_for_node = list(path_list[node].copy())
        
        # Add path from depot (without destination) with inactive flag
        for p in path_for_node[:-1]:
            full_path.append((p, False))
        
        # Reverse path to go back from destination
        path_for_node.reverse()

        # Add reversed path: mark visited nodes with True flag
        for p in (path_for_node[:-1]):
            if p in ungolden_nodes:
                full_path.append((p, True))
                ungolden_nodes.remove(p)
            else:
                full_path.append((p, False))
    
    # Append return to depot
    full_path.append((0, False))
    
    return full_path


def create_population(problem_instance, population_size):
    population = []
    near_list=neighborhood_greedy_strategy_dijistra(problem_instance=problem_instance)
    if problem_instance._density<2:
        for n in range(population_size):
            individual=choice_a_path(near_list, seed=n)
            population.append(individual)
        return population
    else:
        return population



#### MUTATION AND CROSSOVER

To solve this problem i have implemented two different mutation and one crossover. This genetic operators are a bit slow, because works in a smart way to reduce the cost. 
The first mutation try to insert a new node in a random part of a path and go to take the gold from there. The possible node to insert are the ones with the less distance.
The second mutation work on the number of trip i have to do to take the gold. When beta is bigger than 1, divide the gold in more trip reduce significantly the cost. So it try to do more trip on one subpath
The crossover impose a sub-path from one individual on another one. 
After applying mutation and crossover, the algorithm make adjustment to maintain that the gold is taken only one time in the path.
To reduce the total cost, the operators return also how much the new individual cost has changed respect before. In this way the cost function is runned only on small segment of the solution


In [897]:
def find_node_with_flag_true(path, wanted_node):
    # Find index of node with active flag in path
    for i in range(len(path)):
        if path[i][1] and path[i][0] == wanted_node:
            return i
    return -1

def founding_start_and_end_index(path, start_index):
    # Find trip boundaries: start and end indices where node==0
    end_index=start_index+1
    while start_index > 0 and path[start_index][0]!=0:
        start_index-=1
    while end_index < len(path) - 1 and path[end_index][0]!=0:
        end_index+=1

    return start_index, end_index

def find_True_flags(path_segment):
    # Extract all nodes with active flag (True) from segment
    gold_elements=[]
    for node, flag in path_segment:
        if flag:
            gold_elements.append(node)

    return gold_elements

def remove_more_gold_nodes(parent_path, gold_nodes, start_index_2, end_index_2):
    # Remove gold node from path and update boundary indices
    new_path=parent_path.copy()
    # Locate node with active flag
    index=find_node_with_flag_true(new_path, gold_nodes)
    
    # Find trip segment containing node
    start_index, end_index = founding_start_and_end_index(new_path, index)
    list_of_gold_nodes_in_segment = find_True_flags(new_path[start_index:end_index])
    
    # If only gold node in segment, remove entire trip
    if len(list_of_gold_nodes_in_segment)==1:
        new_path=new_path[:start_index]+new_path[end_index:]
        # Adjust boundaries after deletion
        delta=end_index-start_index
        start_index_2=start_index_2-delta if start_index_2 > start_index else start_index_2
        end_index_2=end_index_2-delta if end_index_2 > end_index else end_index_2
        return new_path, start_index_2, end_index_2
    
    # Otherwise deactivate flag only
    new_path[index]=(new_path[index][0], False)

    return new_path, start_index_2, end_index_2


def find_path_removed(parent_path, gold_nodes):
    # Find and return path segment before removing gold node
    new_path=parent_path.copy()
    # Locate node with active flag
    index=find_node_with_flag_true(new_path, gold_nodes)
    
    # Find trip segment containing node
    start_index, end_index = founding_start_and_end_index(new_path, index)
    list_of_gold_nodes_in_segment = find_True_flags(new_path[start_index:end_index])
    old_insert=new_path[start_index:end_index+1]
    
    # If only gold node in segment, remove entire trip
    if len(list_of_gold_nodes_in_segment)==1:
        new_path=new_path[:start_index]+new_path[end_index:]
        new_insert=[]
        return new_path, new_insert, old_insert
    
    # Otherwise deactivate flag and return modified segment
    new_path[index]=(new_path[index][0], False)
    new_insert=new_path[start_index:end_index+1]

    return new_path, new_insert, old_insert


def insert_more_gold_nodes(gold_nodes, path_list):
    # Build path for specific gold nodes using greedy strategy
    full_path = []
    seed = random.randint(0, 1000)
    rng = random.Random(seed)
    ungolden_nodes = gold_nodes.copy()
    
    # Build path by randomly selecting unvisited gold nodes
    while ungolden_nodes:
        # Randomly choose next gold node to visit
        node = rng.choice(ungolden_nodes)
        
        # Get shortest path from depot to chosen node
        path_for_node = list(path_list[node].copy())
        
        # Add path from depot (without destination) with inactive flag
        for p in path_for_node[:-1]:
            full_path.append((p, False))
        
        # Reverse path to go back from destination
        path_for_node.reverse()
        
        # Add reversed path: mark visited gold nodes with True flag
        for p in (path_for_node[:-1]):
            if p in ungolden_nodes:
                full_path.append((p, True))
                ungolden_nodes.remove(p)
            else:
                full_path.append((p, False))
    
    # Append return to depot
    full_path.append((0, False))

    return full_path


In [898]:
def apply_insertion(segment, idx_rel, new_node, graph):
    """
    Inserisci new_node nel segmento dopo il nodo in posizione idx_rel.
    Calcola il percorso più breve da current_node a new_node.
    I nodi intermedi hanno flag=False, il nodo destinazione ha flag=True.
    
    Args:
        segment: lista di (node_id, flag) da modificare
        idx_rel: indice relativo nel segmento dove si trova current_node
        new_node: nodo da inserire
        graph: grafo NetworkX
    
    Returns:
        lista modificata con new_node inserito
    """
    current_node = segment[idx_rel][0]
    
    # Calcola percorso più breve
    path_link = nx.shortest_path(graph, source=current_node, target=new_node, weight='dist')
    
    # Costruisci i nodi intermedi con flag=False (non collezionano oro)
    intermediate_nodes = [(n, False) for n in path_link[1:-1]]
    
    # Inserisci: intermedi + destinazione con flag=True
    new_segment = segment[:idx_rel + 1] + intermediate_nodes + [(new_node, True)] + segment[idx_rel + 1:]
    
    return new_segment


def mutation_neighbor_of_next_insertion_only(path, problem_instance, path_list, neighbor_distance_cache, graph):
    """
    Mutazione semplice e diretta:
    1. Seleziona un nodo con flag=True nel segmento attuale
    2. Cerca i k vicini più prossimi di questo nodo dalla cache
    3. Dalla cache, seleziona uno dei k vicini più prossimi
    4. Filtra candidati: escludi depot, current_node, e nodi già attivi nello STESSO segmento
    5. Inserisci il nuovo nodo tramite apply_insertion()
    6. Rimuovi duplicati: disattiva flag e elimina segmenti vuoti
    
    Args:
        path: percorso da mutare (lista di tuple (node_id, flag))
        problem_instance: istanza del problema
        path_list: lista dei percorsi dalle shortest paths (per compatibilità)
        neighbor_distance_cache: cache dei k vicini per ogni nodo
        graph: grafo NetworkX
    
    Returns:
        (new_path, delta) dove delta = (infeas_change, cost_change)
    """
    new_path = path.copy()
    
    # Scegli un indice casuale nel segmento (escludendo primo e ultimo)
    random_index = random.randint(1, len(new_path) - 2)
    
    # Trova i confini del segmento (dove ritorna al depot)
    start_index, end_index = founding_start_and_end_index(new_path, random_index)
    
    # Salva il segmento originale per il confronto della delta
    starting_segment = path[start_index:end_index + 1]
    
    # Trova il primo nodo con flag=True nel segmento
    idx_to_mutate = None
    for i in range(start_index, end_index):
        if path[i][1]:  # Se flag è True
            idx_to_mutate = i
            break
    
    # Se non trovi nodo attivo, esci
    if idx_to_mutate is None:
        return path, (0, 0)
    
    # Prendi il nodo successivo e il nodo corrente
    next_node = new_path[idx_to_mutate + 1][0]
    current_node = new_path[idx_to_mutate][0]
    
    # === CORREZIONE CRITICA: Estrai i vicini dalla cache per current_node (il nodo da cui partire) ===
    # Se inserisci un nodo tra current_node e next_node, devi cercare i vicini di current_node!
    cached_neighbors = neighbor_distance_cache.get(current_node, {})
    
    if not cached_neighbors:
        return path, (0, 0)
    
    # Estrai i nodi attivi NEL SEGMENTO ATTUALE
    active_nodes_in_segment = {node for node, flag in path[start_index:end_index + 1] if flag}
    
    # Filtra candidati
    candidates = []
    for neighbor in cached_neighbors.keys():
        # Escludi: depot (0), current_node, e nodi già attivi nello STESSO segmento
        # PERMETTI nodi attivi in altri segmenti: la logica di duplicati li gestisce dopo
        if neighbor != 0 and neighbor != current_node and neighbor not in active_nodes_in_segment:
            candidates.append(neighbor)
    
    # Se nessun candidato disponibile, non mutare
    if not candidates:
        return path, (0, 0)
    
    # Scegli casualmente uno dei candidati
    new_node = random.choice(candidates)
    
    # Inserisci il nuovo nodo nel segmento tramite apply_insertion
    new_segment = apply_insertion(
        starting_segment, 
        idx_to_mutate - start_index,  # Adatta l'indice relativo al segmento
        new_node, 
        graph
    )
    
    # Ricostruisci il percorso completo
    new_path = new_path[:start_index] + new_segment + new_path[end_index + 1:]
    
    # === CORREZIONE: Rimuovi duplicati e segmenti vuoti ===
    # Se new_node appare con flag=True OVUNQUE nel percorso (incluso nel segmento modificato), disattivalo
    new_segment_end = start_index + len(new_segment) - 1
    
    # Colleziona gli indici dei segmenti da rimuovere
    segments_to_remove = []
    
    # Cerca duplicati OVUNQUE nel percorso
    i = 0
    while i < len(new_path):
        # Se troviamo new_node con flag=True FUORI dal segmento modificato, è un duplicato
        if new_path[i][0] == new_node and new_path[i][1]:
            if i < start_index or i > new_segment_end:
                # Disattiva il flag per evitare duplicati
                new_path[i] = (new_node, False)
                
                # Trova il segmento (da 0 a 0) che contiene questo nodo
                if i < len(new_path) - 1:  # Proteggi da index out of bounds
                    seg_start, seg_end = founding_start_and_end_index(new_path, i)
                    
                    # Controlla se il segmento contiene ancora nodi con flag=True
                    has_active_nodes = any(new_path[j][1] for j in range(seg_start, min(seg_end + 1, len(new_path))))
                    
                    # Se non ci sono più nodi attivi nel segmento, marcalo per la rimozione
                    if not has_active_nodes:
                        segments_to_remove.append((seg_start, seg_end))
                
                # Uscire dal loop: il nuovo nodo è stato inserito una sola volta,
                # quindi può avere al massimo un duplicato nel percorso
                break
        
        i += 1
    
    # Rimuovi il segmento vuoto (se esiste)
    if segments_to_remove:
        seg_start, seg_end = segments_to_remove[0]
        new_path = new_path[:seg_start] + new_path[seg_end:]
    else:
        # Ricostruisci il segmento in modo pulito (solo se non è stato distrutto)
        # Trova il primo e ultimo nodo con flag=True nel segmento rimasto
        first_index_true = None
        last_index_true = None
        first_element_true = None
        last_element_true = None
        
        for i in range(start_index, end_index + 1):
            if new_path[i][1]:  # Se flag è True
                if first_index_true is None:
                    first_index_true = i
                    first_element_true = new_path[i][0]
                last_index_true = i
                last_element_true = new_path[i][0]
        
        # Se esiste almeno un nodo con flag=True nel segmento
        if first_element_true is not None:
            seg = []
            
            # Parte 1: da 0 al primo elemento True (usando path_list)
            path_to_first = path_list[first_element_true]
            for node in path_to_first[:-1]:  # Escludi il nodo destinazione
                seg.append((node, False))
            seg.append((first_element_true, True))
            
            # Parte 2: la parte intermedia tra first_True e last_True (esclusi i nodi stessi)
            for i in range(first_index_true + 1, last_index_true):
                seg.append(new_path[i])
            
            # Parte 3: il nodo last_True (se diverso da first_True)
            if first_element_true != last_element_true:
                seg.append((last_element_true, True))
            
            # Parte 4: da ultimo elemento True a 0 (usando path_list invertito)
            path_from_last = path_list[last_element_true]
            path_from_last_reversed = list(reversed(path_from_last))
            for node in path_from_last_reversed[1:-1]:  # Escludi last_element_true e 0
                seg.append((node, False))
            seg.append((0, False))
            
            # Sostituisci il segmento in new_path
            new_path = new_path[:start_index] + seg + new_path[end_index + 1:]
        


    
    
    
    
    




    # === CALCOLO DELLA DELTA CORRETTO ===
    # Calcola il costo TOTALE prima e dopo considerando TUTTI i segmenti modificati
    # Costo PRIMA: percorso originale
    infeas_before, cost_before = calculate_full_path_cost_final(problem_instance, path)
    
    # Costo DOPO: percorso con mutazione, duplicati rimossi e segmenti vuoti eliminati
    infeas_after, cost_after = calculate_full_path_cost_final(problem_instance, new_path)
    
    delta_infeas = infeas_after - infeas_before
    delta_cost = cost_after - cost_before
    
    return new_path, (delta_infeas, delta_cost)

In [899]:
def mutate_trip_counts(trip_counts):
    # Multiply selected trip count by 4 to test cost reduction
    
    new_counts = trip_counts[:]  
    # Select random trip segment to modify
    idx_to_change = random.randint(0, len(new_counts) - 1)
    
    # Increase trip count to distribute weight across trips
    increment = 4
    new_counts[idx_to_change] *= increment
    
    return new_counts, idx_to_change


def get_trip_boundaries(path, cache=None):
    # Return trip segment boundaries using cache if available
    if cache is not None:
        return cache.get_trip_boundaries()
    
    # Identify depot returns to find trip boundaries
    boundaries = []
    trip_start = 0
    
    # Mark (start, end) indices for each trip (returns to depot at node 0)
    for i, (node, _) in enumerate(path):
        if node == 0 and i > 0:
            boundaries.append((trip_start, i))
            trip_start = i
    
    # Add final trip if path doesn't end at depot
    if trip_start < len(path):
        boundaries.append((trip_start, len(path) - 1))
    
    return boundaries

def evaluate_trip_mutation_smart(problem_instance, path, old_counts, new_counts, changed_idx):
    # Evaluate cost change for modified trip count on segment only
    boundaries = get_trip_boundaries(path)
    start_idx, end_idx = boundaries[changed_idx]
    
    # Extract only affected trip segment for evaluation
    mini_path = path[start_idx : end_idx + 1]
    
    # Compute costs with old and new trip counts
    old_trip_count_list = [old_counts[changed_idx]]
    new_trip_count_list = [new_counts[changed_idx]]
    
    # Calculate cost delta on sub-path only (efficient evaluation)
    _, cost_old = calculate_full_path_cost_final(problem_instance, mini_path, old_trip_count_list)
    _, cost_new = calculate_full_path_cost_final(problem_instance, mini_path, new_trip_count_list)
    
    delta = cost_new - cost_old
    
    return delta


In [900]:
def crossover_zero_paths_with_delta(parent1, parent2, possible_paths, problem_instance):
    
    p2_working = parent2.copy()
    
    start_index_1 = random.randint(1, len(parent1) - 2)
    start_index_1, end_index_1 = founding_start_and_end_index(parent1, start_index_1)
    
    segment_p1 = parent1[start_index_1 : end_index_1 + 1]
    
    gold_element = 0
    index_gold_element_1 = -1
    for i in range(start_index_1, end_index_1):
        if parent1[i][1] == True:
            gold_element = parent1[i][0]
            index_gold_element_1 = i
            break
            
    if gold_element == 0:
        return p2_working, (0, 0, [])

    list_gold_1 = find_True_flags(parent1[index_gold_element_1+1 : end_index_1])

    index_gold_element_2 = find_node_with_flag_true(p2_working, gold_element)
    
    try:
        start_index_2, end_index_2 = founding_start_and_end_index(p2_working, index_gold_element_2)
    except (IndexError, TypeError):
        return p2_working, (0, 0, [])

    segment_p2_original = p2_working[start_index_2 : end_index_2 + 1]
    
    res_seg1 = calculate_full_path_cost_final(problem_instance, segment_p1)
    res_seg2 = calculate_full_path_cost_final(problem_instance, segment_p2_original)
    
    delta_infeas = res_seg1[0] - res_seg2[0]
    delta_cost = res_seg1[1] - res_seg2[1]
    infeas_nodes = []
    if len(res_seg1) > 2: infeas_nodes.extend(res_seg1[2])

    list_gold_2 = find_True_flags(p2_working[start_index_2 : end_index_2])
    
    if len(list_gold_1) > 0:
        for node in list_gold_1:
            if node not in list_gold_2:
                
                idx_external = find_node_with_flag_true(p2_working, node)
                
                if idx_external != -1:
                    s_ext, e_ext = founding_start_and_end_index(p2_working, idx_external)
                    seg_ext_old = p2_working[s_ext : e_ext + 1]
                    res_ext_old = calculate_full_path_cost_final(problem_instance, seg_ext_old)
                    
                    p2_working, start_index_2, end_index_2 = remove_more_gold_nodes(
                        p2_working, node, start_index_2, end_index_2
                    )
                    
                    check_idx = min(idx_external, len(p2_working)-2)
                    s_ext_new, e_ext_new = founding_start_and_end_index(p2_working, check_idx)
                    seg_ext_new = p2_working[s_ext_new : e_ext_new + 1]
                    res_ext_new = calculate_full_path_cost_final(problem_instance, seg_ext_new)
                    
                    delta_infeas += (res_ext_new[0] - res_ext_old[0])
                    delta_cost += (res_ext_new[1] - res_ext_old[1])
                    if len(res_ext_new) > 2: infeas_nodes.extend(res_ext_new[2])

    if gold_element in list_gold_2:
        list_gold_2.remove(gold_element)
    
    list_2_not_in_1 = [node for node in list_gold_2 if node not in list_gold_1]

    new_path_append = []
    if len(list_2_not_in_1) > 0:
        new_path_append = insert_more_gold_nodes(list_2_not_in_1, possible_paths)
        
        res_append = calculate_full_path_cost_final(problem_instance, new_path_append)
        
        delta_infeas += res_append[0]
        delta_cost += res_append[1]
        if len(res_append) > 2: infeas_nodes.extend(res_append[2])

    new_individual = (
        p2_working[:start_index_2] + 
        segment_p1 +
        p2_working[end_index_2+1:] +
        new_path_append[1:]
    )
    
    return new_individual, (delta_infeas, delta_cost, infeas_nodes)

#### GENETIC ALGORITHM

To make evolve better the algorithm 

In [901]:

def tournament_selection(population_with_fitness, tournament_size=3):
    candidates = random.sample(population_with_fitness, tournament_size)
    
    winner = min(candidates, key=lambda x: (x[1][0], x[1][1]))
    
    return winner[0], winner[1]

In [902]:
def precompute_neighbor_distances(problem_instance):
    graph = problem_instance.graph
    cache = {}
    
    for node in graph.nodes():
        neighbors_with_dist = {}
        node_coords = graph.nodes[node]['pos']
        
        for neighbor in graph.neighbors(node):
            if neighbor != 0:
                n_coords = graph.nodes[neighbor]['pos']
                dist = ((n_coords[0] - node_coords[0])**2 + 
                        (n_coords[1] - node_coords[1])**2) ** 0.5
                neighbors_with_dist[neighbor] = dist
        
        cache[node] = neighbors_with_dist
    
    return cache

In [903]:
def hill_climbing(problem_instance, initial_solution, n_iterations=1000):
    
    current_solution = initial_solution.copy()
    
    curr_infeas, curr_cost= calculate_full_path_cost_final(problem_instance, current_solution)
    
    neighbor_distance_cache = build_nearest_neighbors_cache(problem_instance, k=5)
    path_list = neighborhood_greedy_strategy_dijistra(problem_instance)
    graph = problem_instance.graph
    
    for iteration in range(n_iterations):
        
        neighbor_solution, delta_tuple = mutation_neighbor_of_next_insertion_only(
            current_solution, problem_instance, path_list, neighbor_distance_cache, graph
        )
        
        delta_infeas = delta_tuple[0]
        delta_cost = delta_tuple[1]
        
        accept = False
        
        if delta_infeas < 0:
            accept = True
            
        elif delta_infeas > 0:
            accept = False
            
        else:
            if delta_cost <= 0:
                accept = True
            else:
                accept = False
        
        
        if accept:
            current_solution = neighbor_solution
            curr_infeas += delta_infeas
            curr_cost += delta_cost

    return current_solution, curr_cost

In [904]:
def genetic_algorithm(problem_instance, population_size=100, n_generations=50, temperature=0.8):
    
    # Crea cache basata su dist_dict con i 5 nodi più vicini
    neighbor_distance_cache = build_nearest_neighbors_cache(problem_instance, k=5)
    graph = problem_instance.graph 
    
    raw_population = create_population(problem_instance, population_size)
    population_data = []
    
    for ind in raw_population:
        fit = calculate_full_path_cost_final(problem_instance, ind)
        population_data.append((ind, fit))
        
    path_list = neighborhood_greedy_strategy_dijistra(problem_instance)

    def is_better(fit_a, fit_b):
        if fit_a[0] != fit_b[0]:
            return fit_a[0] < fit_b[0]
        return fit_a[1] < fit_b[1]

    for generation in range(n_generations):
        new_population_data = []
        
        progress = generation / n_generations
        
        min_tournament_k = 2
        max_tournament_k = 6
        current_tournament_k = int(min_tournament_k + progress * (max_tournament_k - min_tournament_k))
        current_tournament_k = max(2, min(6, current_tournament_k))
        
        base_crossover_rate = 0.1
        max_added_rate = 0.8
        current_crossover_prob = base_crossover_rate + (progress * temperature * max_added_rate)
        current_crossover_prob = max(0.0, min(1.0, current_crossover_prob))

        best_of_gen = min(population_data, key=lambda x: (x[1][0], x[1][1]))
        new_population_data.append(best_of_gen)
        
        while len(new_population_data) < population_size:
            parent1_path, p1_fit = tournament_selection(population_data, current_tournament_k)
            parent2_path, p2_fit = tournament_selection(population_data, current_tournament_k)
            
            winner_path = None
            winner_fit = None
            
           
            if random.random() > current_crossover_prob:
                child_path, mut_delta = mutation_neighbor_of_next_insertion_only(
                    parent1_path, problem_instance, path_list, neighbor_distance_cache, graph
                )
                
                m_infeas = p1_fit[0] + mut_delta[0]
                m_cost = p1_fit[1] + mut_delta[1]
                child_fit = (m_infeas, m_cost, [])

                if is_better(child_fit, p1_fit):
                    winner_path, winner_fit = child_path, child_fit
                else:
                    winner_path, winner_fit = parent1_path, p1_fit

            else:
                child_path, delta_tuple = crossover_zero_paths_with_delta(
                    parent1_path, parent2_path, path_list, problem_instance
                )
                
                child_infeas = p2_fit[0] + delta_tuple[0]
                child_cost = p2_fit[1] + delta_tuple[1]
                child_fit = (child_infeas, child_cost, []) 
                
                if is_better(child_fit, p2_fit):
                    winner_path, winner_fit = child_path, child_fit
                else:
                    winner_path, winner_fit = parent2_path, p2_fit
            
            
            new_population_data.append((winner_path, winner_fit))
        
        population_data = new_population_data
        
        if generation % 10 == 0 or generation == n_generations - 1:
            best_now = min(population_data, key=lambda x: (x[1][0], x[1][1]))
            print(f"Gen {generation} | K={current_tournament_k} | CrossoverRate: {current_crossover_prob:.2f} | Best: {best_now[1][1]:.2f}")

    return min(population_data, key=lambda x: (x[1][0], x[1][1]))

In [905]:
def run_hill_climbing_trips(problem_instance, path, initial_trip_counts, max_iter=1000):
    # Initialize trip counts and compute initial cost
    current_counts = list(initial_trip_counts)
    _, current_total_cost = calculate_full_path_cost_final(problem_instance, path, current_counts)
    
    iteration = 0
    improvements = 0
    scaler = 8
    
    # Try different scalers to initialize trip counts efficiently
    for i in range(2):
        # Multiply each element by scaler and convert to int
        new_counts = [int(c * scaler) for c in current_counts]
        _, new_cost = calculate_full_path_cost_final(problem_instance, path, new_counts)
        
        if new_cost < current_total_cost:
            current_counts = new_counts
            current_total_cost = new_cost
        else:
            scaler = scaler / 2
        
    # Hill climbing loop: optimize trip counts
    while iteration < max_iter:
        iteration += 1
        
        # Generate neighbor solution by multiplying random trip count
        new_counts, changed_idx = mutate_trip_counts(current_counts)
        
        # Evaluate cost change on affected segment only
        delta = evaluate_trip_mutation_smart(
            problem_instance, 
            path, 
            current_counts,
            new_counts, 
            changed_idx
        )
        
        # Accept improvement if cost delta is negative
        if delta < 0:
            current_counts = new_counts
            current_total_cost += delta
            improvements += 1

    # Print optimization results
    print(f"\n--- Fine Ottimizzazione ---")
    print(f"Costo Finale: {current_total_cost:.4f}")
    print(f"Miglioramenti trovati: {improvements}")
    print(f"Counts Finali: {current_counts}")
    
    return current_counts, current_total_cost


In [906]:
def conversion_solution(problem_instance,full_path, trip_counts):
    converted_path = []
    list_of_bounds=get_trip_boundaries(full_path)
    for trip_idx in range(len(trip_counts)):
        start_idx, end_idx = list_of_bounds[trip_idx]
        trip_count = trip_counts[trip_idx]
        for num_trips in range(trip_count):
            for i in range(start_idx, end_idx):
                node, is_active = full_path[i]
                if is_active:
                    gold_dict = problem_instance.gold_dict 
                    total_gold = gold_dict[node]
                    gold_amount = total_gold / trip_count if trip_count > 0 else 0
                    converted_path.append((node, gold_amount))
                else:
                    converted_path.append((node, 0))

    converted_path.append((0, 0))
    converted_path=converted_path[1:]
    return converted_path

In [907]:
def solving_problem_with_ga_and_trip_optimization(problem_instance, population_size=100, n_generations=50, n_trip_count_hc=500):
    
    ga_result = genetic_algorithm(problem_instance, population_size, n_generations)
    best_path = ga_result[0]
    
    initial_trip_counts = [1 for _ in range(len(get_trip_boundaries(best_path)))]
    
    optimized_trip_counts, final_cost = run_hill_climbing_trips(
        problem_instance, 
        best_path, 
        initial_trip_counts, 
        max_iter=n_trip_count_hc
    )
    
    return best_path, optimized_trip_counts, final_cost

In [921]:
def adaptive_solver(problem_instance, n_trip_count_hc=500, fast=True):
    
    # Extract number of nodes and ensure minimum threshold
    n = len(problem_instance.graph.nodes())
    n = max(100, n)
    n=min (n,1000)

    # Compute population and generation count based on problem size
    population_size = int((30 - (n - 100) / 60) / 2)
    n_generations = int(80 - (n - 100) / 18)
    n_trip_count_hc=int(n)

    if n>=800:
       n_trip_count_hc= n_trip_count_hc/2

    # Double population for sparse graphs
    if problem_instance._density < 0.8:
        population_size *= 2

    # Reduce generations for large, dense, high-beta problems
    if n > 500 and fast and problem_instance._beta > 1:
        n_generations = int(n_generations / 2)

    # Choose algorithm based on density
    if problem_instance._density < 0.8:
        # Sparse graphs: use genetic algorithm
        ga_result = genetic_algorithm(problem_instance, population_size, n_generations)
        best_path = ga_result[0]
    else:
        # Dense graphs: use hill climbing starting from greedy solution
        path_list = neighborhood_greedy_strategy_dijistra(problem_instance)
        initial_path = choice_a_path(path_list, seed=42)
        best_path, cost = hill_climbing(problem_instance, initial_path, n_iterations=population_size * n_generations)
        print(cost)
    # Initialize trip counts

    initial_trip_counts = [1 for _ in range(len(get_trip_boundaries(best_path)))]
    
    if problem_instance._beta > 1:
        optimized_trip_counts, final_cost = run_hill_climbing_trips(
            problem_instance, 
            best_path, 
            initial_trip_counts, 
            max_iter=n_trip_count_hc
        )
    else:
        optimized_trip_counts = initial_trip_counts
        _, final_cost = calculate_full_path_cost_final(problem_instance, best_path, optimized_trip_counts)
    
    return best_path, optimized_trip_counts, final_cost

In [909]:
k=create_population(P,10)
print(k)

[[(0, False), (50, True), (0, False), (99, True), (0, False), (55, True), (0, False), (6, True), (0, False), (35, True), (0, False), (70, True), (0, False), (67, True), (0, False), (56, True), (0, False), (41, True), (0, False), (69, True), (0, False), (49, True), (0, False), (85, True), (0, False), (29, True), (0, False), (76, True), (0, False), (19, True), (0, False), (42, True), (0, False), (20, True), (0, False), (14, True), (0, False), (97, True), (0, False), (39, True), (0, False), (87, True), (0, False), (98, True), (0, False), (23, True), (0, False), (52, True), (0, False), (15, True), (0, False), (11, True), (0, False), (60, True), (0, False), (83, True), (0, False), (17, True), (0, False), (65, True), (0, False), (80, True), (0, False), (59, True), (0, False), (37, True), (0, False), (92, True), (0, False), (86, True), (0, False), (48, True), (0, False), (4, True), (0, False), (81, True), (0, False), (94, True), (0, False), (57, True), (0, False), (96, True), (0, False), (1, 

In [922]:
P=Problem(800, density=0.7, alpha=1, beta=2)
x=adaptive_solver(P)
print(x)
print(P.baseline())

#funzione che conta quanti True ho
def count_true_flags(path):
    return sum(1 for _, flag in path if flag)

count_true_flags(x[0])

Gen 0 | K=2 | CrossoverRate: 0.10 | Best: 42771518.67
Gen 10 | K=4 | CrossoverRate: 0.42 | Best: 42568576.71
Gen 19 | K=5 | CrossoverRate: 0.71 | Best: 42473852.97

--- Fine Ottimizzazione ---
Costo Finale: 540834.2780
Miglioramenti trovati: 223
Counts Finali: [64, 256, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 256, 64, 256, 64, 64, 64, 64, 64, 64, 256, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 256, 64, 256, 64, 64, 64, 64, 64, 64, 64, 64, 64, 256, 64, 256, 256, 64, 256, 256, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 256, 64, 64, 64, 256, 256, 64, 64, 64, 64, 64, 64, 256, 64, 256, 64, 256, 64, 64, 256, 64, 64, 64, 256, 64, 64, 64, 64, 64, 64, 64, 256, 64, 64, 64, 256, 64, 64, 256, 64, 64, 256, 64, 256, 64, 256, 64, 256, 64, 64, 256, 64, 64, 64, 64, 256, 64, 256, 64, 256, 256, 256, 256, 256, 64, 64, 64, 256, 256, 256, 256, 256, 64, 64, 256, 64, 64, 256, 64, 256, 256, 256, 64, 256, 256, 64, 64, 256, 64, 64, 64, 64, 256, 64, 64, 64, 256, 256, 64, 256, 64, 64, 64, 64, 64, 64, 256, 25

799

In [911]:
calculate_full_path_cost_final(P,x[0])

(0, np.float64(18261.138463367348))

In [912]:
P.baseline()

np.float64(18266.18579582672)

In [913]:
def validate_solution_uniqueness(path):
    """
    Valida che ogni nodo con flag=True appaia una sola volta nella soluzione.
    
    Args:
        path: lista di tuple (node_id, flag)
    
    Returns:
        (is_valid, violations) dove:
        - is_valid: True se tutti i nodi con True appaiono una sola volta
        - violations: lista di nodi che appaiono più di una volta con True
    """
    nodes_with_true = []
    
    # Estrai tutti i nodi con flag=True
    for node, flag in path:
        if flag:
            nodes_with_true.append(node)
    
    # Conta le occorrenze
    from collections import Counter
    node_counts = Counter(nodes_with_true)
    
    # Trova i nodi che appaiono più di una volta
    violations = [node for node, count in node_counts.items() if count > 1]
    
    is_valid = len(violations) == 0
    
    return is_valid, violations


def print_solution_validation(path, context=""):
    """
    Stampa un avviso se la soluzione ha nodi con flag=True duplicati.
    Se è valida, non stampa nulla (silenzioso al successo).
    
    Args:
        path: lista di tuple (node_id, flag)
        context: stringa di contesto per il messaggio (es. "HILL CLIMBING", "GENETIC ALGORITHM")
    """
    is_valid, violations = validate_solution_uniqueness(path)
    
    if not is_valid:
        print("\n" + "="*100)
        print(f"⚠️  ATTENZIONE: NODI DUPLICATI CON FLAG=TRUE IN {context}")
        print("="*100)
        print(f"\nNodi che appaiono più di una volta con flag=True: {violations}")
        
        # Mostra le posizioni dei duplicati
        for node in violations:
            positions = [i for i, (n, flag) in enumerate(path) if n == node and flag]
            print(f"  Nodo {node}: posizioni {positions}")
        
        print("\n" + "="*100 + "\n")
    
    return is_valid


In [ ]:

# === VALUTAZIONE COMPLETA DELL'ADAPTIVE SOLVER ===
import pandas as pd
import time

print("="*120)
print("VALUTAZIONE ADAPTIVE SOLVER - COMPREHENSIVE TEST")
print("="*120)

# Parametri di test
densities = [0.3, 0.5, 0.8, 1.0]  # 4 densità
sizes = [100, 300, 600, 1000]      # 4 grandezze
betas = [0.5, 1.0, 2.0]             # 3 beta

results = []

total_problems = len(densities) * len(sizes) * len(betas)
current_problem = 0

print(f"\nRisoluzione di {total_problems} problemi...\n")

for density in densities:
    for size in sizes:
        for beta in betas:
            current_problem += 1
            
            # Crea il problema
            P = Problem(size, density=density, alpha=1.0, beta=beta, seed=42)
            
            # Calcola la baseline
            baseline_cost = P.baseline()
            
            # Risolvi con adaptive_solver
            start_time = time.time()
            best_path, trip_counts, solver_cost = adaptive_solver(P, n_trip_count_hc=500, fast=True)
            elapsed_time = time.time() - start_time
            
            # Calcola il miglioramento
            improvement = ((baseline_cost - solver_cost) / baseline_cost) * 100
            
            # Salva i risultati
            results.append({
                'Density': density,
                'Size': size,
                'Beta': beta,
                'Baseline': baseline_cost,
                'Solver': solver_cost,
                'Improvement (%)': improvement,
                'Time (s)': elapsed_time
            })
            
            # Stampa progresso
            print(f"[{current_problem:2d}/{total_problems}] Size={size:4d}, Density={density:.1f}, Beta={beta:.1f} | "
                  f"Baseline={baseline_cost:10.2f}, Solver={solver_cost:10.2f}, "
                  f"Improvement={improvement:6.2f}%, Time={elapsed_time:6.2f}s")

print("\n" + "="*120)
print("RISULTATI COMPLETI")
print("="*120)

# Crea un DataFrame
df = pd.DataFrame(results)

# Formatta i risultati
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("\n" + df.to_string(index=False))

# Statistiche di riepilogo
print("\n" + "="*120)
print("STATISTICHE DI RIEPILOGO")
print("="*120)
print(f"\nTempo medio per problema: {df['Time (s)'].mean():.2f}s")
print(f"Tempo totale: {df['Time (s)'].sum():.2f}s")
print(f"\nMiglioramento medio: {df['Improvement (%)'].mean():.2f}%")
print(f"Miglioramento minimo: {df['Improvement (%)'].min():.2f}%")
print(f"Miglioramento massimo: {df['Improvement (%)'].max():.2f}%")

print("\n" + "="*120)
print("PERFORMANCE PER DENSITÀ")
print("="*120)
by_density = df.groupby('Density')[['Improvement (%)', 'Time (s)']].agg(['mean', 'min', 'max'])
print(by_density)

print("\n" + "="*120)
print("PERFORMANCE PER GRANDEZZA")
print("="*120)
by_size = df.groupby('Size')[['Improvement (%)', 'Time (s)']].agg(['mean', 'min', 'max'])
print(by_size)

print("\n" + "="*120)
print("PERFORMANCE PER BETA")
print("="*120)
by_beta = df.groupby('Beta')[['Improvement (%)', 'Time (s)']].agg(['mean', 'min', 'max'])
print(by_beta)

print("\n" + "="*120)
